# Implmentation of LSTM and GRU

The maximum accuracy achived on MNIST for both LSTM and GRU is 97%. 


### Learning curve for GRU and LSTM

In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable
from torch.nn import Parameter
from torch import Tensor
import torch.nn.functional as F

import math


cuda = True if torch.cuda.is_available() else False
    
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor    

torch.manual_seed(125)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(125)


In [2]:
import numpy as np
import pandas as pd
# from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import torch
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset


class KddData(object):

    def __init__(self, batch_size, file_name1, file_name2):
        data1 = pd.read_csv(file_name1)
        data2 = pd.read_csv(file_name2)
        
        
        self._encoder = {
            'label':    LabelEncoder()
        }
        self.batch_size = batch_size

        target = np.array(data1['label'])
        features = np.array(data1.drop('label', axis=1))
        data_X, data_y = self.__encode_data(features, target)
        self.train_dataset = TensorDataset(
            torch.from_numpy(data_X.astype(np.float32)),
            torch.from_numpy(data_y.astype(np.int64))
        )

        target = np.array(data2['label'])
        features = np.array(data2.drop('label', axis=1))
        data_X, data_y = self.__encode_data(features, target)
        self.test_dataset = TensorDataset(
            torch.from_numpy(data_X.astype(np.float32)),
            torch.from_numpy(data_y.astype(np.int64))
        )

 
        self.train_dataloader = DataLoader(self.train_dataset, self.batch_size, shuffle=True)
        self.test_dataloader = DataLoader(self.test_dataset, self.batch_size, shuffle=True)

    """将数据中字符串部分转换为数字，并将输入的41维特征转换为8*8的矩阵"""
    def __encode_data(self, data_X, data_y):
        self._encoder['label'].fit(list(set(data_y)))
        data_X = np.pad(data_X, ((0, 0), (0, 100 - len(data_X[0]))), 'constant').reshape(-1, 1, 10, 10)
        data_y = self._encoder['label'].transform(data_y)
        return data_X, data_y

    """将数据拆分为训练集和测试集，并转换为TensorDataset对象"""
    def __split_data_to_tensor(self, data_X, data_y):
        X_train, X_test, y_train, y_test = train_test_split(data_X, data_y, test_size=0.3)
        train_dataset = TensorDataset(
            torch.from_numpy(X_train.astype(np.float32)),
            torch.from_numpy(y_train.astype(np.int64))
        )
        test_dataset = TensorDataset(
            torch.from_numpy(X_test.astype(np.float32)),
            torch.from_numpy(y_test.astype(np.int64))
        )
        return train_dataset, test_dataset

    """接受一个数组进行解码"""
    def decode(self, data, label=False):
        if not label:
            _data = list(data)
            _data[1] = self._encoder['protocal'].inverse_transform([_data[1]])[0]
            _data[2] = self._encoder['service'].inverse_transform([_data[2]])[0]
            _data[2] = self._encoder['flag'].inverse_transform([_data[3]])[0]
            return _data
        return self._encoder['label'].inverse_transform(data)
    
    def encode(self, data, label=False):
        if not label:
            _data = list(data)
            _data[1] = self._encoder['protocal'].transform([_data[1]])[0]
            _data[2] = self._encoder['service'].transform([_data[2]])[0]
            _data[3] = self._encoder['flag'].transform([_data[3]])[0]
            return _data
        return self._encoder['label'].transform([data])[0]


# data_file = 'test-remote-nagle-open-cn2us.csv'
# data_file = 'test-cn2cn-nagle-open-wired.csv'
# dataset = KddData(batch_size, 'train-expon-nagle-open.csv', data_file)
# dataset.train_dataset[0][0].shape

In [3]:
'''
STEP 1: LOADING DATASET
'''

batch_size = 64

trainFile = 'train-wired.csv'
testFile = 'test-wired.csv'
testFile1 = 'test-kr.csv'
testFile2 = 'test-us.csv'
testFile3 = 'test-wifi.csv'
testFile4 = 'test-4G.csv'
testFile5 = 'test-3G.csv'


dataset = KddData(batch_size, trainFile, testFile)
 
n_iters = 12000
num_epochs = n_iters / (len(dataset.train_dataset) / batch_size)
num_epochs = int(num_epochs)

In [4]:
train_loader = dataset.train_dataloader
 
test_loader = dataset.test_dataloader

## LSTM cell implementation

In [7]:
class LSTMCell(nn.Module):

    """
    An implementation of Hochreiter & Schmidhuber:
    'Long-Short Term Memory' cell.
    http://www.bioinf.jku.at/publications/older/2604.pdf

    """

    def __init__(self, input_size, hidden_size, bias=True):
        super(LSTMCell, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.bias = bias
        self.x2h = nn.Linear(input_size, 4 * hidden_size, bias=bias)
        self.h2h = nn.Linear(hidden_size, 4 * hidden_size, bias=bias)
        self.reset_parameters()



    def reset_parameters(self):
        std = 1.0 / math.sqrt(self.hidden_size)
        for w in self.parameters():
            w.data.uniform_(-std, std)
    
    def forward(self, x, hidden):
        
        hx, cx = hidden
        
        x = x.view(-1, x.size(1))
        
        gates = self.x2h(x) + self.h2h(hx)
    
        gates = gates.squeeze()
        
        ingate, forgetgate, cellgate, outgate = gates.chunk(4, 1)
        
        ingate = F.sigmoid(ingate)
        forgetgate = F.sigmoid(forgetgate)
        cellgate = F.tanh(cellgate)
        outgate = F.sigmoid(outgate)
        

        cy = torch.mul(cx, forgetgate) +  torch.mul(ingate, cellgate)        

        hy = torch.mul(outgate, F.tanh(cy))
        
        return (hy, cy)


In [8]:
'''
STEP 3: CREATE MODEL CLASS
'''
 
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim, bias=True):
        super(LSTMModel, self).__init__()
        # Hidden dimensions
        self.hidden_dim = hidden_dim
         
        # Number of hidden layers
        self.layer_dim = layer_dim
               
        self.lstm = LSTMCell(input_dim, hidden_dim, layer_dim)  
        
        self.fc = nn.Linear(hidden_dim, output_dim)
     
    
    
    def forward(self, x):
        
        # Initialize hidden state with zeros
        #######################
        #  USE GPU FOR MODEL  #
        #######################
        #print(x.shape,"x.shape")100, 28, 28
        if torch.cuda.is_available():
            h0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).cuda())
        else:
            h0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim))

        # Initialize cell state
        if torch.cuda.is_available():
            c0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).cuda())
        else:
            c0 = Variable(torch.zeros(self.layer_dim, x.size(0), hidden_dim))

                    
       
        outs = []
        
        cn = c0[0,:,:]
        hn = h0[0,:,:]

        for seq in range(x.size(1)):
            hn, cn = self.lstm(x[:,seq,:], (hn,cn)) 
            outs.append(hn)
            
    

        out = outs[-1].squeeze()
        
        out = self.fc(out) 
        # out.size() --> 100, 10
        return out
 

In [9]:

input_dim = 10
hidden_dim = 128
layer_dim = 1  # ONLY CHANGE IS HERE FROM ONE LAYER TO TWO LAYER
output_dim = 2
 
model = LSTMModel(input_dim, hidden_dim, layer_dim, output_dim)
# model = GRUModel(input_dim, hidden_dim, layer_dim, output_dim)

#######################
#  USE GPU FOR MODEL  #
#######################
 
if torch.cuda.is_available():
    model.cuda()

criterion = nn.CrossEntropyLoss()

learning_rate = 0.1
 
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  


LSTM local

In [ ]:
# Number of steps to unroll
seq_dim = 10 

loss_list = []
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Load images as Variable
        #######################
        #  USE GPU FOR MODEL  #
        #######################
          
        if torch.cuda.is_available():
            images = Variable(images.view(-1, seq_dim, input_dim).cuda())
            labels = Variable(labels.cuda())
        else:
            images = Variable(images.view(-1, seq_dim, input_dim))
            labels = Variable(labels)
          
        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()
         
        # Forward pass to get output/logits
        # outputs.size() --> 100, 10
        outputs = model(images)

        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)

        if torch.cuda.is_available():
            loss.cuda()

        # Getting gradients w.r.t. parameters
        loss.backward()

        # Updating parameters
        optimizer.step()
        
        loss_list.append(loss.item())
        iter += 1
         
        if iter % 500 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in test_loader:
                #######################
                #  USE GPU FOR MODEL  #
                #######################
                if torch.cuda.is_available():
                    images = Variable(images.view(-1, seq_dim, input_dim).cuda())
                else:
                    images = Variable(images.view(-1 , seq_dim, input_dim))
                
                # Forward pass only to get logits/output
                outputs = model(images)
                
                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)
                 
                # Total number of labels
                total += labels.size(0)
                 
                # Total correct predictions
                #######################
                #  USE GPU FOR MODEL  #
                #######################
                if torch.cuda.is_available():
                    correct += (predicted.cpu() == labels.cpu()).sum()
                else:
                    correct += (predicted == labels).sum()
             
            accuracy = 100 * correct / total
             
            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))

In [12]:
import torchmetrics


def runmodel(dataset):
    accuracy = torchmetrics.Accuracy()
    f1score = torchmetrics.F1(average='macro', num_classes=2)
    eval_loss = 0
    eval_acc = 0

    test_loader = dataset.test_dataloader
    for images, labels in test_loader:
        #######################
        #  USE GPU FOR MODEL  #
        #######################
        if torch.cuda.is_available():
            images = Variable(images.view(-1, seq_dim, input_dim).cuda())
        else:
            images = Variable(images.view(-1 , seq_dim, input_dim))
        # Forward pass only to get logits/output
        outputs = model(images)

        # Get predictions from the maximum value
        _, predicted = torch.max(outputs.data, 1)
        
        
        labels = labels.cpu()
        predicted = predicted.cpu()
        accuracy(predicted, labels)
        f1score(predicted, labels)

        # label = label.cpu().detach().numpy()
        # pred = pred.cpu().detach().numpy()
        # for i in range(len(label)):
        #     if label[i] == 1:
        #         mal_count += 1
        #         if pred[i] == 1:
        #             mal_correct +=1
        #     else:
        #         benign_count += 1
        #         if pred[i] == 1:
        #             benign_error += 1
        
    # print('malicious recall:{:.6f}, false Positive:{:.6f}'.format(mal_correct/mal_count, benign_error/()))
    acc = accuracy.compute().data.detach()
    print('Accuracy:',acc)
    f1sc = f1score.compute().data.detach()
    print(f" f1score: {f1sc}")

trainFile = 'train-wired.csv'

testFile = 'test-wired.csv'
testFile1 = 'test-kr.csv'
testFile2 = 'test-us.csv'
testFile3 = 'test-wifi.csv'
testFile4 = 'test-4G.csv'
testFile5 = 'test-3G.csv'


dataset = KddData(batch_size, trainFile, testFile2)
runmodel(dataset)

Accuracy: tensor(0.5730)
 Recall: 0.5043684840202332, precision: 0.5067330002784729, f1score: 0.48113584518432617
